In [3]:
def parse_cigar(cigar_string):
    """Parse a CIGAR string into operation-count pairs."""
    operations = []
    current_count = ""
    
    for char in cigar_string:
        if char.isdigit():
            current_count += char
        else:
            operations.append((char, int(current_count)))
            current_count = ""
    
    return operations

def get_sequence_lengths(cigar_string):
    """
    Calculate original and resulting sequence lengths from CIGAR string.
    Original length: sum of M operations
    Result length: sum of M and N operations
    """
    operations = parse_cigar(cigar_string)
    
    original_length = 0
    result_length = 0
    
    for op, count in operations:
        if op == 'M':
            original_length += count
            result_length += count
        elif op == 'N':
            result_length += count
            
    return original_length, result_length

def create_alignment_matrix(cigar_string):
    """
    Create a dynamic programming alignment matrix from a CIGAR string.
    Only handles M (match/mismatch) and N (skip) operations.
    
    Args:
        cigar_string: CIGAR string with only M and N operations
        
    Returns:
        2D list representing the alignment matrix where:
        1 indicates aligned positions (M)
        0 indicates unaligned positions (N)
    """
    operations = parse_cigar(cigar_string)
    original_length, result_length = get_sequence_lengths(cigar_string)
    
    # Initialize matrix with zeros
    # Add 1 to dimensions for 0-based alignment position
    matrix = [[0 for _ in range(result_length + 1)] for _ in range(original_length + 1)]
    
    # Current positions in sequences
    orig_pos = 0  # position in original sequence
    res_pos = 0   # position in result sequence
    
    # Process each operation in the CIGAR string
    for op, count in operations:
        if op == 'M':
            # For matches/mismatches, mark diagonal positions
            for i in range(count):
                orig_pos += 1
                res_pos += 1
                matrix[orig_pos][res_pos] = 1
        elif op == 'N':
            # For skips, only advance in the result sequence
            res_pos += count
        else:
            raise ValueError(f"Unsupported operation: {op}")
    
    return matrix

def print_matrix(matrix):
    """Pretty print the alignment matrix."""
    for row in matrix:
        print(' '.join(str(x) for x in row))
        
cigar = "2M1N3M"  # Example CIGAR string
matrix = create_alignment_matrix(cigar)
print_matrix(matrix)

0 0 0 0 0 0 0
0 1 0 0 0 0 0
0 0 1 0 0 0 0
0 0 0 0 1 0 0
0 0 0 0 0 1 0
0 0 0 0 0 0 1


In [5]:
cigar = "2M1N3M1N2M"
introns = [4]

matrix = create_alignment_matrix(cigar)
print_matrix(matrix)

0 0 0 0 0 0 0 0 0 0
0 1 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0
0 0 0 0 1 0 0 0 0 0
0 0 0 0 0 1 0 0 0 0
0 0 0 0 0 0 1 0 0 0
0 0 0 0 0 0 0 0 1 0
0 0 0 0 0 0 0 0 0 1


In [1]:
def parse_cigar(cigar_string):
    """Parse a CIGAR string into operation-count pairs."""
    operations = []
    current_count = ""
    
    for char in cigar_string:
        if char.isdigit():
            current_count += char
        else:
            operations.append((char, int(current_count)))
            current_count = ""
    
    return operations

def realign_cigar(cigar_string, ref_introns):
    """
    Realign a CIGAR string to match reference intron positions.
    
    Args:
        cigar_string: Original CIGAR string (M and N operations only)
        ref_introns: List of reference intron positions (1-based)
        
    Returns:
        New CIGAR string with introns at reference positions
    """
    # Parse original CIGAR
    operations = parse_cigar(cigar_string)
    
    # Calculate sequence lengths
    total_matches = sum(count for op, count in operations if op == 'M')
    intron_lengths = [count for op, count in operations if op == 'N']
    
    # Validate reference intron positions
    for pos in ref_introns:
        if pos < 1 or pos >= total_matches:
            raise ValueError(f"Invalid intron position {pos}. Must be between 1 and {total_matches-1}")
    
    # Sort reference intron positions
    ref_introns_sorted = sorted(ref_introns)
    
    # Create new CIGAR string
    new_cigar = []
    
    # Initialize variables
    current_pos = 0
    intron_index = 0
    
    for ref_pos in ref_introns_sorted:
        # Add matches before the intron
        matches_before = ref_pos - current_pos
        if matches_before > 0:
            new_cigar.append(f"{matches_before}M")
            current_pos += matches_before
        
        # Add the intron
        if intron_index < len(intron_lengths):
            new_cigar.append(f"{intron_lengths[intron_index]}N")
            intron_index += 1
    
    # Add remaining matches after the last intron
    remaining_matches = total_matches - current_pos
    if remaining_matches > 0:
        new_cigar.append(f"{remaining_matches}M")
    
    return "".join(new_cigar)

# Test cases
cigar = "4M2N3M1N2M"
ref_introns = [5]

new_cigar = realign_cigar(cigar, ref_introns)
print(f"Original CIGAR: {cigar}")
print(f"New CIGAR: {new_cigar}")

Original CIGAR: 4M2N3M1N2M
New CIGAR: 5M2N4M


In [54]:
matrix = create_alignment_matrix(cigar)
print_matrix(matrix)

0 0 0 0 0 0 0 0 0 0 0 0 0
0 1 0 0 0 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0 0 0 0
0 0 0 1 0 0 0 0 0 0 0 0 0
0 0 0 0 1 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 1 0 0 0 0 0
0 0 0 0 0 0 0 0 1 0 0 0 0
0 0 0 0 0 0 0 0 0 1 0 0 0
0 0 0 0 0 0 0 0 0 0 0 1 0
0 0 0 0 0 0 0 0 0 0 0 0 1


In [56]:
new_matrix = create_alignment_matrix(new_cigar)
print_matrix(new_matrix)

0 0 0 0 0 0 0 0 0 0
0 1 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0
0 0 0 1 0 0 0 0 0 0
0 0 0 0 1 0 0 0 0 0
0 0 0 0 0 1 0 0 0 0
0 0 0 0 0 0 1 0 0 0
0 0 0 0 0 0 0 1 0 0
0 0 0 0 0 0 0 0 1 0
0 0 0 0 0 0 0 0 0 1


In [ ]:
# what if we evaluate each intron separately? Introducing one intron at a time, superimposing two matrices and moving on to the next intron